# Calculate proportions for Airdrop

In [1]:
import json
from pprint import pprint
import pandas as pd
df = pd.read_csv('data/pills-snapshot.csv')
df.head(1)

,Unnamed: 0,address,pills_erc20_balance_historical,lqdr_pills_balance,pills_erc20_balance_current,neo_pools_balance,beets_pills_balance
0,0,0x9C775D3D66167685B2A3F4567B548567D2875350,2.415931e+06,0.0,281012.897854,0.0,0.0


We begin with the total circulating supply of PILLS at the time of the snapshot.

In [2]:
df['pills_erc20_balance_historical'].sum()

6714972.014113384

Out of the circulating supply, a substantial proportion of PILLS were transferred to contracts.
Any PILLS in a contract (Neo pools, Spirit+LQDR, Beets) were therefore no longer controlled by a "wallet" address.
We've inspected those specific contracts to connect wallet addresses with the PILLS staked in those contracts.
We will remove those contracts from the snapshot; they are not wallet addresses and we've attributed their balances to real wallets.

In [3]:
# remove Spirit LP balance
df = df[df.address != '0x9C775D3D66167685B2A3F4567B548567D2875350']

# remove all Neo Pools
with open('data/neo-pools.json', 'r') as f:
    neo_pool_addresses = json.load(f)
neo_pool_addresses.reverse()
neo_pool_addresses = [ i['address'] for i in neo_pool_addresses ]
df = df[~df.address.isin(neo_pool_addresses)]

# the Neo Pools balance in any pool prior to the continuous Neo Pools
other_neo_pool_total = 336780.609907302

# remove Beets balance
df = df[df.address != '0x20dd72Ed959b6147912C2e529F0a0C651c33c9ce']

pills_for_airdrop = df['pills_erc20_balance_historical'].sum() + \
    df['neo_pools_balance'].sum() + \
    df['lqdr_pills_balance'].sum() + \
    df['beets_pills_balance'].sum()

Total PILLS controlled by an address is given by the sum of all columns except pills_erc20_balance_current.
Double-check that the total matches circulating supply.

In [4]:
df['total'] = df['pills_erc20_balance_historical'] + \
    df['neo_pools_balance'] + \
    df['lqdr_pills_balance'] + \
    df['beets_pills_balance']

df['total'].sum() + other_neo_pool_total

6714972.014113386

Compute proportions for airdrop. Double-check that proportion assigned is 100%.

In [5]:
df['proportion'] = df['total'] / pills_for_airdrop
df['proportion'].sum()

1.0

Apply verified address changes by request.

In [6]:
df_address_changes = pd.read_csv('data/address-changes.csv')
address_changes = dict(zip(df_address_changes.old, df_address_changes.new))
df['address'] = df['address'].map(address_changes).fillna(df.address)

In [7]:
df_proportion = df[['address', 'proportion']]
df_proportion.to_csv('data/airdrop.csv', index=False)
df_proportion.head()

,address,proportion
1,0xF04588329c3104779796F279A9df3803Ae83dB7B,7.942674e-03
3,0x2728e32174FC95De00695007c9134fA03dC8b434,8.368566e-03
4,0xD488F4585Ab28D003324253be5D791aC103c0b57,1.368727e-22
5,0x4BC4Da7839a3c987D44F0DBDAcBDd1f53243Ba24,0.000000e+00
6,0xD3743Ec801f157FBfF93FcC813fdBA231026cBF2,0.000000e+00


Prepare CSV for use with disperse.app

In [8]:
df_disperse = df.copy()
df_disperse = df_disperse[['address', 'proportion']]

# allocate 100_000 MPX to disperse
df_disperse['token'] = df_disperse['proportion'] * 100_000
del(df_disperse['proportion'])

# remove zero-balances and dust balances
df_disperse = df_disperse[df_disperse.token > 0.01]
df_disperse.to_csv('data/disperse.csv', index=False, header=False)
df_disperse.head(5)

,address,token
1,0xF04588329c3104779796F279A9df3803Ae83dB7B,794.267399
3,0x2728e32174FC95De00695007c9134fA03dC8b434,836.856639
8,0xE1e96F427297C1A7c597cAEDF127D7906F1f2794,0.012758
10,0xe17A3d8399ebc55CC9C6b5719687B70D5fbf44a4,291.252617
16,0xb41243FD8f0aA4D2Cf22598788588d1bC62b883c,0.401218


At current quote 1 MPX == 0.617 USD, what is USD value of dust that was removed?

In [9]:
(100_000 - df_disperse['token'].sum()) * 0.617

0.33731877241955954

It will cost more gas to disperse dust than the total value of that dust.
Similarly, dust recipients would pay more in gas for even a single transaction than dust is worth.

Double-check that amount to be dispersed is 100_000 (minus dust)

In [10]:
df_disperse['token'].sum()

99999.45329210305